Author: Omar El Malki (omar.elmalki@epfl.ch)

### ConceptNet Node extraction from first 10 ROCStories

In [4]:
import pandas as pd
import os
from tqdm import tqdm
import ast

tqdm.pandas()

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

In [5]:
os.chdir('../../src/')

FileNotFoundError: [Errno 2] No such file or directory: '../../src/'

In [6]:
# Read ROCStories into pandas DataFrame
roc_stories_path_csv = "../generated/semantic-role-labeling/ROCStories_resolved_with_entities_first10.csv"
roc_stories_df = pd.read_csv(roc_stories_path_csv, sep='\t', header=0)

#### Turn entity list into strings before applying CoCo-Ex

In [19]:
def entities_to_string(entities):
    """
    Transform list of entities into a single string
    :param triples:
    :return: str
    """
    s = ""
    l = ast.literal_eval(entities)
    for entity in l:
        s += entity + ", "
    return s

In [20]:
roc_stories_entities_df = roc_stories_df[['storyid', 'srl_entities1', 'srl_entities2', 'srl_entities3', 'srl_entities4', 'srl_entities5']]

In [21]:
roc_stories_entities_df

,storyid,srl_entities1,srl_entities2,srl_entities3,srl_entities4,srl_entities5
0,8bbe6d11-1e2e-413c-bf81-eaea05f4f1bd,"['David', 'David had put on a lot of weight recently', 'David', 'a lot of weight']","['David', 'David habits', 'David', 'David', 'the reason']","['David', ""David'd been eating too much fast food lately"", ""David'd"", 'too much fast food']","['David', 'going to burger places', 'David', 'to burger places', 'David', 'a vegetarian diet']","['to feel much better', 'David', 'much better']"
1,0beabab2-fb49-460e-a6e6-f35a202e3348,"['Tom', 'a very short temper']","['a guest', 'Tom very angry']","['Tom', 'a hole in the wall of Tom house']","[""Tom 's guest"", 'afraid', ""Tom 's guest""]","['Tom', 'on Tom couch', 'Tom', 'with regret about Tom actions']"
2,87da1a22-df0b-410c-b186-439700b70ba6,"['Marcus', 'clothing']","['All of Marcus clothes', 'either too formal or too casual']","['Marcus', 'to buy a pair of khakis', 'Marcus', 'a pair of khakis']",[],"['Marcus', 'happy to have the right clothes for the event', 'Marcus', 'the right clothes for the event']"
3,2d16bcd6-692a-4fc0-8e7c-4a6f81d9efa9,"['Bobby', 'Bill should buy a trailer and haul a trailer with Bill car', 'Bill', 'buy a trailer', 'Bill', 'a trailer', 'Bill', 'a trailer']","['Bill', 'a truck would be better for what Bill needed', 'a truck', 'better for what Bill needed', 'what', 'Bill']","['Bobby', 'two vehicles were much more expensive', 'two vehicles', 'much more expensive']","['Bill', 'in Bill ways']","['Bill', 'buying a truck', 'Bill', 'a truck']"
4,c71bb23b-7731-4233-8298-76ba6886cee1,"['John', 'a pastor with a very bad memory']","['John', 'to memorize John sermons many days in advance', 'John', 'John sermons', 'John']","['John', 'to learn to sing to overcome John handicap', 'John', 'to sing', 'John', 'John', 'John handicap']","['John', 'all John sermons', 'into music', 'John', 'John', 'all his sermons']","['John congregation', 'delighted', 'so', 'John']"
5,4d7b022e-25d2-4300-a9b0-24ab35f4045b,"[""Melody 's parents"", 'Melody', 'with a trip to the big aquarium']","['Melody', 'a nap']","[""Melody 's parents"", 'Melody', 'energetic and excited']","['Melody', 'sharks , tropical fish and many others']","['Melody and Melody family', 'home']"
6,8036c905-f23e-4976-83a1-85d679b5e0c2,"['The math teacher', 'a pop quiz', 'class']","['some students', 'The math teacher', 'passing out a pop quiz', 'The math teacher', 'a pop quiz']","['I', 'my pencil', 'I', 'to work', 'I']",['I'],"['I', 'I', 'confident', 'I', 'my pencil']"
7,77338898-07d4-4143-8451-284540c8b082,"['My first girlfriend', 'i']",['My first girlfriend'],"['we', 'each other']","['My first girlfriend', 'with me']","['we', ""we could n't be apart so My first girlfriend moved in with me"", 'we', 'apart', 'My first girlfriend']"
8,110fafd1-2bb7-4ffe-aac7-475706165d41,"['I', 'Charlie Horse', 'I', 'four years old']","['Charlie Horse', 'a brown stuffed horse', 'I']","['Charlie Horse', 'my best friend', 'Charlie Horse', 'at the head of my bed']","['I', 'Charlie Horse', 'next to me', 'I', 'Charlie Horse', 'soft fur']","['I', 'to listen to my radio as I fell asleep cuddling Charlie Horse', 'I', 'to my radio', 'I', 'asleep', 'I', 'Charlie Horse']"
9,13573c2e-5eed-40eb-bbe5-ed259b5c76a6,"['Laura', 'corn']","['Laura', 'to grow some in Laura backyard', 'Laura', 'some']","['them', 'The whole process of growing them', 'Laura very excited']","['Laura', 'that them required too much water', 'them', 'too much water']","['Laura', 'Laura corn garden idea']"


In [22]:
for n in range(1, 6):
    roc_stories_entities_df[f'entities_str_t{n}'] = roc_stories_entities_df[f'srl_entities{n}'] \
        .progress_apply(lambda t: entities_to_string(t))

100%|██████████| 10/10 [00:00<00:00, 13851.73it/s]


In [23]:
roc_stories_entities_string_df = roc_stories_entities_df[
    ['storyid', 'entities_str_t1', 'entities_str_t2', 'entities_str_t3', 'entities_str_t4', 'entities_str_t5']]

In [24]:
roc_stories_entities_string_df.to_csv(
    '../generated/conceptnet-node-extraction/ROCStories_resolved_entities_entity_extraction_input_first10.csv', sep='\t', header=None)

In [25]:
roc_stories_entities_string_df

,storyid,entities_str_t1,entities_str_t2,entities_str_t3,entities_str_t4,entities_str_t5
0,8bbe6d11-1e2e-413c-bf81-eaea05f4f1bd,"David, David had put on a lot of weight recently, David, a lot of weight,","David, David habits, David, David, the reason,","David, David'd been eating too much fast food lately, David'd, too much fast food,","David, going to burger places, David, to burger places, David, a vegetarian diet,","to feel much better, David, much better,"
1,0beabab2-fb49-460e-a6e6-f35a202e3348,"Tom, a very short temper,","a guest, Tom very angry,","Tom, a hole in the wall of Tom house,","Tom 's guest, afraid, Tom 's guest,","Tom, on Tom couch, Tom, with regret about Tom actions,"
2,87da1a22-df0b-410c-b186-439700b70ba6,"Marcus, clothing,","All of Marcus clothes, either too formal or too casual,","Marcus, to buy a pair of khakis, Marcus, a pair of khakis,",,"Marcus, happy to have the right clothes for the event, Marcus, the right clothes for the event,"
3,2d16bcd6-692a-4fc0-8e7c-4a6f81d9efa9,"Bobby, Bill should buy a trailer and haul a trailer with Bill car, Bill, buy a trailer, Bill, a trailer, Bill, a trailer,","Bill, a truck would be better for what Bill needed, a truck, better for what Bill needed, what, Bill,","Bobby, two vehicles were much more expensive, two vehicles, much more expensive,","Bill, in Bill ways,","Bill, buying a truck, Bill, a truck,"
4,c71bb23b-7731-4233-8298-76ba6886cee1,"John, a pastor with a very bad memory,","John, to memorize John sermons many days in advance, John, John sermons, John,","John, to learn to sing to overcome John handicap, John, to sing, John, John, John handicap,","John, all John sermons, into music, John, John, all his sermons,","John congregation, delighted, so, John,"
5,4d7b022e-25d2-4300-a9b0-24ab35f4045b,"Melody 's parents, Melody, with a trip to the big aquarium,","Melody, a nap,","Melody 's parents, Melody, energetic and excited,","Melody, sharks , tropical fish and many others,","Melody and Melody family, home,"
6,8036c905-f23e-4976-83a1-85d679b5e0c2,"The math teacher, a pop quiz, class,","some students, The math teacher, passing out a pop quiz, The math teacher, a pop quiz,","I, my pencil, I, to work, I,","I,","I, I, confident, I, my pencil,"
7,77338898-07d4-4143-8451-284540c8b082,"My first girlfriend, i,","My first girlfriend,","we, each other,","My first girlfriend, with me,","we, we could n't be apart so My first girlfriend moved in with me, we, apart, My first girlfriend,"
8,110fafd1-2bb7-4ffe-aac7-475706165d41,"I, Charlie Horse, I, four years old,","Charlie Horse, a brown stuffed horse, I,","Charlie Horse, my best friend, Charlie Horse, at the head of my bed,","I, Charlie Horse, next to me, I, Charlie Horse, soft fur,","I, to listen to my radio as I fell asleep cuddling Charlie Horse, I, to my radio, I, asleep, I, Charlie Horse,"
9,13573c2e-5eed-40eb-bbe5-ed259b5c76a6,"Laura, corn,","Laura, to grow some in Laura backyard, Laura, some,","them, The whole process of growing them, Laura very excited,","Laura, that them required too much water, them, too much water,","Laura, Laura corn garden idea,"


In [26]:
os.chdir('../lib/CoCo-Ex/')

In [27]:
entity_extraction_script_name = 'CoCo-Ex_entity_extraction.py'
input_csv = "../../generated/conceptnet-node-extraction/ROCStories_resolved_entities_entity_extraction_input_first10.csv"
output_tsv = "../../generated/conceptnet-node-extraction/ROCStories_resolved_entities_entity_extraction_output_first10.tsv"
os.system(f'python3 {entity_extraction_script_name} {input_csv} {output_tsv}')

0

In [28]:
output_filtered_tsv = "../../generated/conceptnet-node-extraction/ROCStories_resolved_entities_conceptnet_nodes_filtered_first10.tsv"
overhead_filter_script_name = 'CoCo-Ex_overhead_filter.py'
len_diff_tokenlevel = 1
len_diff_charlevel = 10
dice_coefficient = 0.8

In [29]:
os.system(f'python3 {overhead_filter_script_name} '
          f'--inputfile {output_tsv} '
          f'--outputfile {output_filtered_tsv} '
          f'--len_diff_tokenlevel {len_diff_tokenlevel} '
          f'--len_diff_charlevel {len_diff_charlevel} '
          f'--dice_coefficient {dice_coefficient}')

0

In [30]:
roc_stories_extracted_nodes_df = pd.read_csv(output_filtered_tsv, sep='\t', header=None,
                                             names=['storyid', 'sentence_index', 'sentence', 'nodes'])

In [31]:
roc_stories_extracted_nodes_df

,storyid,sentence_index,sentence,nodes
0,text:0,sent:1,"david, david had put on a lot of weight recently, david, a lot of weight,",[lot][david][weight]
1,text:0,sent:2,"david, david habits, david, david, the reason,",[david][habit][reason]
2,text:0,sent:3,"david, david'd been eating too much fast food lately, david'd, too much fast food,",[food][david]
3,text:0,sent:4,"david, going to burger places, david, to burger places, david, a vegetarian diet,",[place][diet][david][vegetarian diet][vegetarian][burger]
4,text:0,sent:5,"to feel much better, david, much better,",NaN
5,text:1,sent:1,"tom, a very short temper,",[tom][temper]
6,text:1,sent:2,"a guest, tom very angry,",[guest]
7,text:1,sent:3,"tom, a hole in the wall of tom house,",[tom][wall][house][hole]
8,text:1,sent:4,"tom 's guest, afraid, tom 's guest,",[tom][guest]
9,text:1,sent:5,"tom, on tom couch, tom, with regret about tom actions,",[tom][couch][action][regret]
